In [1]:
# Inicializar el ambiente
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
import statsmodels.api as sm


np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1
import warnings
warnings.filterwarnings("ignore")

Cargar el archivo en formato CSV con pandas

In [3]:
df = pd.read_csv('AAC.csv' )#, prefix='X') 
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x11,x12,x13,x14,x15,x16,x17,x18,x19,class
0,6.23608,3.89755,3.11804,4.12027,5.45657,6.01336,2.44989,7.01559,4.67706,11.02450,...,3.45212,5.67929,3.34076,3.45212,9.13140,6.45880,7.57238,1.78174,3.45212,1
1,6.34744,3.89755,3.22940,4.12027,5.34521,6.01336,2.33853,7.12695,4.67706,11.13590,...,3.22940,5.56793,3.45212,3.56347,9.13140,6.34744,7.46102,1.78174,3.56347,1
2,7.50000,3.26087,4.89130,4.67391,4.13043,5.21739,2.60870,7.06522,2.39130,10.00000,...,5.00000,4.13043,2.60870,5.65217,8.15217,7.06522,8.15217,1.95652,3.58696,1
3,7.13624,3.05839,3.61446,6.85820,6.85820,6.20945,2.13160,7.50695,4.17053,8.89713,...,4.54124,4.54124,4.72660,4.54124,9.08248,5.00463,5.37535,1.66821,2.59500,1
4,7.13624,3.05839,3.52178,6.85820,6.85820,6.30213,2.03892,7.41427,4.17053,8.89713,...,4.63392,4.54124,4.72660,4.54124,9.08248,5.00463,5.46803,1.66821,2.68767,1


In [11]:
df.dtypes

x0       float64
x1       float64
x2       float64
x3       float64
x4       float64
x5       float64
x6       float64
x7       float64
x8       float64
x9       float64
x10      float64
x11      float64
x12      float64
x13      float64
x14      float64
x15      float64
x16      float64
x17      float64
x18      float64
x19      float64
class      int64
dtype: object

In [4]:
n_neighbors = 5
num_features = df.shape[1]-1
num_class = 7
class_ = 'class'

dataset = df.values
X = dataset[:,0:num_features]#.astype(float)

#convertir clases categoricas 'string' a discretas 'entero'
clases = pd.DataFrame(df[class_].astype('category'))
clases = clases.apply(lambda x: x.cat.codes)
Y = clases.as_matrix()

In [15]:
Y[:]

array([[0],
       [0],
       [0],
       ...,
       [6],
       [6],
       [6]], dtype=int8)

### Modelo 

In [5]:
seed = 9
np.random.seed(seed)

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

#print(Y)


cv_real = []
cv_pred = []
for train_index, test_index in kfold.split(X, Y):

    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    kmeans = KMeans(n_clusters=7, random_state=seed)
    kmeans.fit(X_train)
    
    #print("cluster_centers_: ", kmeans.cluster_centers_)


    # predict
    y_pred = kmeans.predict(X_test)
    cv_real.extend(y_test)
    cv_pred.extend(y_pred)

    #learning metrics
    #cm = confusion_matrix(y_test, y_pred)
    #print ("Confusion Matrix")
    #print (cm)

    acc = accuracy_score(y_test, y_pred)
    #print ("Accuracy score: %f" % accuracy_score(y_test, y_pred))

cm = confusion_matrix(cv_real[:], cv_pred[:])
print ("Confusion Matrix")
print (cm)
print("Var : ",np.var(cv_pred))
print ("Accuracy score: %f" % accuracy_score(cv_real, cv_pred))
print("---------------------------------------------------------------------<>\n\n\n")

Confusion Matrix
[[12  8  4 13  3  4  2]
 [28 35 18 41 14 32 25]
 [39 57 40 60 29 50 46]
 [23 14  1 14 16 15  8]
 [68 64 46 55 44 78 17]
 [16  9  6 10 14  8  2]
 [49 46 57 43 24 66 19]]
Var :  3.854092350376536
Accuracy score: 0.123563
---------------------------------------------------------------------<>



